# Geração de instâncias

In [1]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB
from udgp import M1, M2, M4, M5, Instance

In [2]:
env = gp.Env()

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2372514
Academic license - for non-commercial use only - registered to braun@cos.ufrj.br


In [3]:
# seed: 1234567
instance = Instance.artificial_molecule(6, freq=False, seed=1234567)
# instance = Instance.lj_cluster(20, freq=True)
# instance = Instance.c60(freq=True)
instance.view_input()#bg_color="#FAFAFA", alpha=1, width=800, height=800)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

# Testes

In [4]:
instance.reset()
m = M4(
        instance, 
        # nx=4,
        env=env,
        # relaxed=True,
    )
m.optimize(log=True)
print(instance.is_solved())
print(m.Runtime)
instance.view()

Set parameter LogToConsole to value 1
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i9-10885H CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Academic license - for non-commercial use only - registered to braun@cos.ufrj.br
Optimize a model with 1875 rows, 975 columns and 5070 nonzeros
Model fingerprint: 0x939ec84a
Model has 15 quadratic constraints
Variable types: 525 continuous, 225 integer (225 binary)
Semi-Variable types: 225 continuous, 0 integer
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [5e-03, 3e+00]
  RHS range        [6e-01, 7e+00]
Presolve removed 270 rows and 30 columns
Presolve time: 0.03s
Presolved: 2250 rows, 1230 columns, 10167 nonzeros
Presolved model has 60 bilinear constraint(s)
Variable types: 780 continuous, 450 integer (450 binary)

Root relaxation

KeyboardInterrupt: 

Exception ignored in: 'gurobipy.logcallbackstub'
Traceback (most recent call last):
  File "c:\Users\Usuario\AppData\Local\Programs\Python\Python311\Lib\site-packages\ipykernel\iostream.py", line 610, in write
    def write(self, string: str) -> Optional[int]:  # type:ignore[override]

KeyboardInterrupt: 


 91082 38607    1.00000   67   45          -    1.00000      -   136  181s


 93165 39438    1.00000   59   47          -    1.00000      -   136  186s


In [6]:
t = []

# for i in range(5):
for seed in [1234567, 123456]:
    print(seed)
    instance = Instance.artificial_molecule(6, freq=False, seed=seed)
    m = M1(
        instance, 
        # nx=4, 
        env=env,
        relaxed=True,
    )
    # if seed == 1234567:
    #     m.SolutionLimit = 1
    m.optimize(log=True)
    print(instance.is_solved())
    t.append(m.Runtime)

np.mean(t)


1234567
Set parameter LogToConsole to value 1
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i9-10885H CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Academic license - for non-commercial use only - registered to braun@cos.ufrj.br
Optimize a model with 75 rows, 975 columns and 570 nonzeros
Model fingerprint: 0x6421540a
Model has 225 quadratic objective terms
Model has 690 quadratic constraints
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [5e-01, 3e+00]
  RHS range        [1e+00, 1e+00]
  QRHS range       [2e+00, 1e+01]
Presolve removed 15 rows and 15 columns

Continuous model is non-convex -- solving as a MIP

Presolve added 135 rows and 0 columns
Presolve removed 0 rows and 135 columns
Presolve time: 0.

KeyboardInterrupt: 

Exception ignored in: 'gurobipy.logcallbackstub'
Traceback (most recent call last):
  File "c:\Users\Usuario\AppData\Local\Programs\Python\Python311\Lib\site-packages\ipykernel\iostream.py", line 610, in write
    def write(self, string: str) -> Optional[int]:  # type:ignore[override]

KeyboardInterrupt: 


 1128888 179573 infeasible  160       -14.68970  -15.00000  2.11%  68.1 1720s
 1131349 180340  -14.69178  178  127  -14.68970  -15.00000  2.11%  68.0 1725s


KeyboardInterrupt: 

Exception ignored in: 'gurobipy.logcallbackstub'
Traceback (most recent call last):
  File "c:\Users\Usuario\AppData\Local\Programs\Python\Python311\Lib\site-packages\ipykernel\iostream.py", line 610, in write
    def write(self, string: str) -> Optional[int]:  # type:ignore[override]

KeyboardInterrupt: 


 1138738 182181  -14.77282  157  149  -14.68970  -15.00000  2.11%  67.9 1738s
 1141115 182984     cutoff  155       -14.68970  -15.00000  2.11%  67.9 1742s


In [6]:
instance.is_solved()

False

In [14]:
print(instance.dists)
print(instance.freqs)

[1.12 1.15 1.82]
[11  0  0]


# Heurística

In [9]:
from IPython.display import clear_output

instance.reset()

previous_a = []

time = 0

i = 1
while not instance.is_solved():
# while instance.fixed_n < instance.n:
    clear_output(wait=True)
    print("=================")
    print(f"TENTATIVA {i}")
    print("=================")
    i += 1

    instance.reset()
    m_core = M4(instance, nx=4, env=env)
    for a_indices in previous_a:
        m_core.addConstr(
            gp.quicksum(m_core.a[i, j, k] for i, j, k in a_indices) 
            <= len(a_indices) - 1
        )
    m_core.optimize()

    if m.Status == GRB.INFEASIBLE:
        previous_a = []

    time += m_core.Runtime

    previous_a.append(instance.a_indices.tolist())
    
    # instance.reset_with_core("mock")
    
    print("CORE")

    broken = False
    while instance.fixed_n < instance.n and not broken:
        print(f"  {instance.fixed_n} átomos")

        solved = False
        tries = 0

        while not solved:
            tries += 1
    
            m = M4(instance, nx=1, env=env)

            solved = m.optimize()
        
            time += m.Runtime
    
            if m.Status == GRB.INFEASIBLE or tries > 20:
                broken = True
                # previous_a.append([(i, j, k) for i, j, k in instance.a_indices])
                break

print(time)

instance.view()


TENTATIVA 1
CORE
  5 átomos
  6 átomos
  7 átomos
  8 átomos
  9 átomos
  10 átomos
  11 átomos
  12 átomos
